In [1]:
import pandas as pd
import sqlite3

In [32]:
# Import the data
df_agencia_br = pd.read_excel('noticias_agencia_brasil.xlsx')

df_brasil_de_fato = pd.read_excel('noticias_brasil_de_fato.xlsx')

df_cbc = pd.read_excel('noticias_CBC.xlsx')

# Concatenate the dataframes using the columns
df = pd.concat([df_agencia_br, df_brasil_de_fato, df_cbc], axis=0, ignore_index=True)
df

,Palavra-chave,URL,Título,Data de Publicação,Conteúdo
0,tarifas,http://agenciabrasil.ebc.com.br/economia/notic...,Dólar cai pela sexta vez seguida e fecha em R$...,25/04/2025 - 19:26,Em meio a sinais de arrefecimento na guerra co...
1,tarifas,http://agenciabrasil.ebc.com.br/internacional/...,Canadenses vão às urnas em eleição dominada po...,28/04/2025 - 13:00,Os eleitores canadenses estão indo às urnas ne...
2,tarifas,http://agenciabrasil.ebc.com.br/internacional/...,Califórnia contesta em tribunal política prote...,17/04/2025 - 10:11,"A Califórnia, estado norte-americano liderado ..."
3,tarifas,http://agenciabrasil.ebc.com.br/economia/notic...,Inflação desacelera para todas as faixas de re...,17/04/2025 - 16:06,A inflação desacelerou em março para todas as ...
4,tarifas,http://agenciabrasil.ebc.com.br/geral/noticia/...,Tarifaço e fundo ambiental são prioridades de ...,26/04/2025 - 16:42,"Enfrentar a crise comercial e tarifária, e pre..."
...,...,...,...,...,...
804,Trump tariffs Brazil,https://www.cbc.ca/news/canada/toronto/ford-us...,"Ontario rips up Starlink deal, plans to tax el...",05/03/2025,Ontario will rip up its $100-million deal with...
805,Trump tariffs Brazil,https://www.cbc.ca/news/canada/newfoundland-la...,Auto part retailers in N.L. have been preparin...,08/04/2025,While the Ontario auto sector might be the tar...
806,Trump tariffs Brazil,https://www.cbc.ca/news/canada/new-brunswick/h...,N.B. may have dodged bullet with Trump tariff ...,02/04/2025,Just two hours after U.S. President Donald Tru...
807,Trump tariffs Brazil,https://www.cbc.ca/news/canada/saskatchewan/tr...,"Uncertainty reigns as Sask. farmers, business ...",03/04/2025,"Saskatchewan farmers, business leaders and tra..."


In [38]:
# Function to update the database using sqlite3, if the database already exists, it will be updated with the new data
def update_database(df, db_name='noticias_database.db'):
    # Connect to the SQLite database (it will be created if it doesn't exist)
    conn = sqlite3.connect(db_name)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Drop the table if it exists to avoid duplicates (optional)
    cursor.execute('DROP TABLE IF EXISTS noticias')

    # Create a new table with the same structure as the DataFrame
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS noticias (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            palavra_chave TEXT,
            url TEXT,
            titulo TEXT,
            data_publicacao TEXT,
            conteudo TEXT
        )
''')

    # Insert the DataFrame data into the table
    for index, row in df.iterrows():
        cursor.execute('''
            INSERT INTO noticias (palavra_chave, url, titulo, data_publicacao, conteudo)
            VALUES (?, ?, ?, ?, ?)
        ''', (row['Palavra-chave'], row['URL'], row['Título'], row['Data de Publicação'], row['Conteúdo']))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

# Call the function to update the database
update_database(df)

In [39]:
# Import the data from the SQLite database into a pandas dataframe
def import_data_from_db(db_name):
    # Create a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    
    # Read the data into a pandas dataframe
    df = pd.read_sql_query("SELECT * FROM noticias", conn)
    
    # Close the connection
    conn.close()
    
    df = df.set_index('id', drop=True)
    return df

df_final = import_data_from_db('noticias_database.db')
df_final

,palavra_chave,url,titulo,data_publicacao,conteudo
id,,,,,
1,tarifas,http://agenciabrasil.ebc.com.br/economia/notic...,Dólar cai pela sexta vez seguida e fecha em R$...,25/04/2025 - 19:26,Em meio a sinais de arrefecimento na guerra co...
2,tarifas,http://agenciabrasil.ebc.com.br/internacional/...,Canadenses vão às urnas em eleição dominada po...,28/04/2025 - 13:00,Os eleitores canadenses estão indo às urnas ne...
3,tarifas,http://agenciabrasil.ebc.com.br/internacional/...,Califórnia contesta em tribunal política prote...,17/04/2025 - 10:11,"A Califórnia, estado norte-americano liderado ..."
4,tarifas,http://agenciabrasil.ebc.com.br/economia/notic...,Inflação desacelera para todas as faixas de re...,17/04/2025 - 16:06,A inflação desacelerou em março para todas as ...
5,tarifas,http://agenciabrasil.ebc.com.br/geral/noticia/...,Tarifaço e fundo ambiental são prioridades de ...,26/04/2025 - 16:42,"Enfrentar a crise comercial e tarifária, e pre..."
...,...,...,...,...,...
805,Trump tariffs Brazil,https://www.cbc.ca/news/canada/toronto/ford-us...,"Ontario rips up Starlink deal, plans to tax el...",05/03/2025,Ontario will rip up its $100-million deal with...
806,Trump tariffs Brazil,https://www.cbc.ca/news/canada/newfoundland-la...,Auto part retailers in N.L. have been preparin...,08/04/2025,While the Ontario auto sector might be the tar...
807,Trump tariffs Brazil,https://www.cbc.ca/news/canada/new-brunswick/h...,N.B. may have dodged bullet with Trump tariff ...,02/04/2025,Just two hours after U.S. President Donald Tru...
